###

# GOR Method Implementation

In [ ]:
import pandas as pd

1. Build a GOR propensity matrix from the files in the propensity folder (for H,E and '-' categories)
2. Create inference method to predict sites in protein sequence via GOR method
3. Compare against secondary structure sequences from PDB

## Create GOR Propensity Matrices

### Create Frequency Tables

In [ ]:
## Code Goes Here
## Output is three tables: one each for H, E, and '-'
## Each table has column for each position relative to the focal residue and a row for each AA


### Create Information Table From Frequency Table

In [ ]:
# Code Goes Here

## Inference Method

In [ ]:
# Code Goes Here
        

## Check Against PDB Files

In [ ]:
# Code Goes Here